In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('beer.txt', sep='\t', header=None)
df.shape
df

,0,1,2,3,4,5,6,7,8,9
0,41.721239,0.503276,2.628182,ale,4.015385,16.73,10.452789,93,13.44,55.337143
1,42.429204,0.525512,1.776364,ale,4.092308,16.72,10.999526,103,12.24,58.380000
2,45.880531,0.443233,2.628182,ale,4.276923,16.68,13.456368,178,10.92,58.382857
3,45.305310,0.471668,1.806364,ale,4.126154,18.84,9.202737,166,10.92,58.525714
4,38.977876,0.392846,2.272727,ale,4.015385,16.77,9.457895,44,10.56,58.900000
...,...,...,...,...,...,...,...,...,...,...
149,39.951327,0.272587,5.217273,stout,4.412308,16.68,6.008368,54,6.84,74.757143
150,40.039823,0.347919,2.859091,stout,4.178462,17.58,7.340842,56,8.88,74.887143
151,43.977876,0.266770,3.775455,stout,4.347692,19.10,7.356000,132,7.32,75.894286
152,45.349558,0.255530,1.301818,stout,4.160000,18.17,3.243579,169,10.68,76.182857


In [3]:
X = df.drop(columns=[3,7], axis='columns')
y = df[3]
global y_class_unique 
y_class_unique = list(y.unique())

In [4]:
def min_max(data):
    new_data = data
    for i in range(data.shape[1]):
        d_temp = data.iloc[:,i]
        d = d_temp.reset_index(drop=True)
        max_data = max(d)
        min_data = min(d)
        for j in range(len(d)):
            new_data.iloc[j,i] = ((d[j] - min_data)/(max_data - min_data))
    return new_data

def standard(data):
    new_data = data.copy()
    for i in range(data.shape[1]):
        d_temp = data.iloc[:,i]
        d = d_temp.reset_index(drop=True)
        mean = np.mean(d)
        sd = np.std(d)
        for j in range(len(d)):
            new_data.iloc[j,i] = (d[j] - mean)/sd
    return new_data

In [5]:
def hypothesis(h_theta, h_X):
    return 1 / (1 + np.exp(-(np.dot(h_theta, h_X.T)))) - 0.0000001

In [6]:
def cost(X, y, theta):
    y1 = hypothesis(X, theta)
    return -(1/len(X)) * np.sum(y*np.log(y1) + (1-y)*np.log(1-y1)) 

In [18]:
def gradient_descent(gd_X, gd_y, gd_theta, gd_alpha, gd_epochs):
    m = len(gd_X)
    cost_list = []
    index = range(0, gd_epochs)
    col_name = [0,1,2]
    cost_df = pd.DataFrame(index=index, columns=col_name)
    cost_df = cost_df.fillna(0)
    counter = 0
    for i in range(0, gd_epochs):
        for j in range(0, gd_y.shape[1]):
            cost_list = []
            gd_theta = pd.DataFrame(gd_theta)
            
            h = hypothesis(gd_theta.iloc[:,j], gd_X)
            for k in range(0, gd_theta.shape[0]):
                
                gd_theta.iloc[k, j] -= (gd_alpha/m) * np.sum((h-gd_y.iloc[:, j])*gd_X.iloc[:, k])
            gd_theta = pd.DataFrame(gd_theta)
            cost_df.iloc[i, j] = cost(gd_X, gd_y.iloc[:, j], gd_theta.iloc[:,j])
    return gd_theta, cost_df

In [8]:
def fit(fit_x, fit_y, fit_alpha, fit_epochs):
    
    new_fit_y = pd.get_dummies(fit_y)
    fit_theta = np.zeros([fit_x.shape[1], new_fit_y.shape[1]])
    theta_fit, cost = gradient_descent(fit_x, new_fit_y, fit_theta, fit_alpha, fit_epochs)
    return theta_fit, cost

In [9]:
def accuracy(accuracy_x, accuracy_y, accuracy_theta):
    original_y = []
    predicted_y = predict(accuracy_x, accuracy_theta)
    accuracy = 0
    accuracy_y = accuracy_y.reset_index(drop=True)
    for i in range(len(predicted_y)):
        if accuracy_y[i] == predicted_y[i]:
            accuracy += 1
    return accuracy/len(accuracy_x)

In [10]:
def predict(predict_x, predict_theta):
    output = []
    for i in range(0, 3):
        h = hypothesis(predict_theta.iloc[:,i], predict_x)
        output.append(h)
    output=pd.DataFrame(output)
    result = []
    for i in range(output.shape[1]):
        temp = output.iloc[:,i]
        max_t = temp.max()
        temp = list(temp)
        result.append(temp.index(max_t))
    y_unique = y_class_unique
    y_unique.sort()
    new_result = []
    for i in range(len(result)):
        new_result.append(y_unique[result[i]])
    return new_result

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train = standard(X_train)
X_test = standard(X_test)

In [19]:
f, cost_value = fit(X_train, y_train, 0.02, 1500)

In [ ]:
fig = cost_value.plot(kind='line',  figsize=(20, 16), title = "cost reduction graph", fontsize=20).get_figure()
fig.savefig('cost.jpeg')

In [ ]:
round(accuracy(X_test, y_test, f), 2)

In [ ]:
predicted_result = (predict(X_test, f))
predicted_result = pd.DataFrame(predicted_result, y_test)
predicted_result.to_csv("predicted_result.csv", index_label = ['predicted', 'original'], mode = 'a')

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1500)

In [ ]:
def get_score(X_train, X_test, y_train, y_test):
    fit_result, cost_score = fit(X_train, y_train, 0.02, 1500)
    return round(accuracy(X_test, y_test, fit_result), 2)

In [ ]:
scores_logistic = []
sklearn_lr = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train = standard(X_train)
    X_test = standard(X_test)
    model.fit(X_train, y_train)
    sklearn_lr.append(round(model.score(X_test,y_test), 2))
    scores_logistic.append(get_score(X_train, X_test, y_train, y_test))

In [ ]:
print("Implemented Algorithm : ", scores_logistic)
print("")
print("Sklearn Algorithm : ", sklearn_lr)

In [ ]:
print("Implemented algorithm : ", round(np.mean(scores_logistic), 2))
print("Sklearn algorithm : ", np.mean(sklearn_lr))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

y_predicted = predict(X_test, f)
confusion = confusion_matrix(y_test, y_predicted, labels = ['ale', 'lager', 'stout'])

sn.set(font_scale=1.4)
ax = sn.heatmap(confusion, annot=True, annot_kws={"size": 16}) # font size

ax.set_xticklabels(['ale', 'lager', 'stout'], rotation = 45, fontsize = 15)
ax.set_yticklabels(['ale', 'lager', 'stout'], rotation = 25, fontsize = 15)
ax.set_xlabel("actual values")
ax.set_ylabel("predicted values")

plt.show()